In [20]:
# As usual, a bit of setup
import time, os, json
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [29]:
def load_Pandora18K(data_dir = '../dataset/Pandora_18k_flat_and_resized'):
    
    i = 0
    
    for root, dirs, files in os.walk(data_dir):
        path = root.split(os.sep)
        for file in files:
            if '.jpg' in str(file).lower(): # an image file
                file_path = os.path.join(root,file)

                imagedata = imread(file_path)
                h,w,c = imagedata.shape
                print(str(root))
                print(str(file) + "\n")
                
                i = i + 1
                
                if i > 100:                
                    return
            #else: # a style folder
                
                
    
    #return X_data, y_data
load_Pandora18K()

../dataset/Pandora_18k_flat_and_resized/01_Byzantin_Iconography
ikon0710.jpg

../dataset/Pandora_18k_flat_and_resized/01_Byzantin_Iconography
ikon0770.jpg

../dataset/Pandora_18k_flat_and_resized/01_Byzantin_Iconography
ikon0761.jpg

../dataset/Pandora_18k_flat_and_resized/01_Byzantin_Iconography
ikon0358.jpg

../dataset/Pandora_18k_flat_and_resized/01_Byzantin_Iconography
ikon0887.jpg

../dataset/Pandora_18k_flat_and_resized/01_Byzantin_Iconography
ikon1779.jpg

../dataset/Pandora_18k_flat_and_resized/01_Byzantin_Iconography
ikon0590.jpg

../dataset/Pandora_18k_flat_and_resized/01_Byzantin_Iconography
ikon1590.jpg

../dataset/Pandora_18k_flat_and_resized/01_Byzantin_Iconography
ikon0708.jpg

../dataset/Pandora_18k_flat_and_resized/01_Byzantin_Iconography
ikon1759.jpg

../dataset/Pandora_18k_flat_and_resized/01_Byzantin_Iconography
ikon0627.jpg

../dataset/Pandora_18k_flat_and_resized/01_Byzantin_Iconography
ikon1824.jpg

../dataset/Pandora_18k_flat_and_resized/01_Byzantin_Iconography


In [5]:
def get_data(num_training=49000, num_validation=1000, num_test=10000):
    data_dir = '../dataset/Pandora_18k_flat_and_resized'
    X_train, y_train, X_test, y_test = load_Pandora18K(data_dir)

    # Subsample the data
    mask = range(num_training, num_training + num_validation)
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = range(num_training)
    X_train = X_train[mask]
    y_train = y_train[mask]
    mask = range(num_test)
    X_test = X_test[mask]
    y_test = y_test[mask]

    return X_train, y_train, X_val, y_val, X_test, y_test

# Invoke the above function to get our data.
X_train, y_train, X_val, y_val, X_test, y_test = get_data()
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

2

In [17]:
# clear old variables
tf.reset_default_graph()

# setup input (e.g. the data that changes every batch)
# The first dim is None, and gets sets automatically based on batch size fed in
X = tf.placeholder(tf.float32, [None, 500, 500, 3])
y = tf.placeholder(tf.int64, [None])
is_training = tf.placeholder(tf.bool)

def style_model(img, y):
    """Generate class scores for img
    
    Inputs:
    - img: Tensor of image [batch_size, 500, 500, 3]
    - y: class [batch_size, 18]
    
    Returns:
    Class scores [batch_size, 18].
    """
    
    dims = tf.shape(img)
    batch_size = dims[0]
    
    conv1 = tf.layers.conv2d(
        inputs=img,
        filters=32,
        kernel_size=[5, 5],
        padding="valid",
        activation=tf.nn.relu)

    print(conv1)
    
    flat = tf.reshape(conv1, [batch_size, 7872512]) #496 * 496 * 32
    
    print(flat)
    
    dense = tf.layers.dense(inputs=flat, units=18, activation=tf.nn.relu)

    return dense

y_out = style_model(X, y)

# define our loss
total_loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=y_out)
mean_loss = tf.reduce_mean(total_loss)

# define our optimizer
optimizer = tf.train.AdamOptimizer(5e-4) # select optimizer and set learning rate
train_step = optimizer.minimize(mean_loss)


Tensor("conv2d/Relu:0", shape=(?, 496, 496, 32), dtype=float32)
Tensor("Reshape:0", shape=(?, 7872512), dtype=float32)


In [18]:
def run_model(session, predict, loss_val, Xd, yd,
              epochs=1, batch_size=64, print_every=100,
              training=None, plot_losses=False):
    # have tensorflow compute accuracy
    correct_prediction = tf.equal(tf.argmax(predict,1), y)
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    # shuffle indicies
    train_indicies = np.arange(Xd.shape[0])
    np.random.shuffle(train_indicies)

    training_now = training is not None
    
    # setting up variables we want to compute (and optimizing)
    # if we have a training function, add that to things we compute
    variables = [mean_loss,correct_prediction,accuracy]
    if training_now:
        variables[-1] = training
    
    # counter 
    iter_cnt = 0
    for e in range(epochs):
        # keep track of losses and accuracy
        correct = 0
        losses = []
        # make sure we iterate over the dataset once
        for i in range(int(math.ceil(Xd.shape[0]/batch_size))):
            # generate indicies for the batch
            start_idx = (i*batch_size)%X_train.shape[0]
            idx = train_indicies[start_idx:start_idx+batch_size]
            
            # create a feed dictionary for this batch
            feed_dict = {X: Xd[idx,:],
                         y: yd[idx],
                         is_training: training_now }
            # get batch size
            actual_batch_size = yd[i:i+batch_size].shape[0]
            
            # have tensorflow compute loss and correct predictions
            # and (if given) perform a training step
            loss, corr, _ = session.run(variables,feed_dict=feed_dict)
            
            # aggregate performance stats
            losses.append(loss*actual_batch_size)
            correct += np.sum(corr)
            
            # print every now and then
            if training_now and (iter_cnt % print_every) == 0:
                print("Iteration {0}: with minibatch training loss = {1:.3g} and accuracy of {2:.2g}"\
                      .format(iter_cnt,loss,np.sum(corr)/actual_batch_size))
            iter_cnt += 1
        total_correct = correct/Xd.shape[0]
        total_loss = np.sum(losses)/Xd.shape[0]
        print("Epoch {2}, Overall loss = {0:.3g} and accuracy of {1:.3g}"\
              .format(total_loss,total_correct,e+1))
        if plot_losses:
            plt.plot(losses)
            plt.grid(True)
            plt.title('Epoch {} Loss'.format(e+1))
            plt.xlabel('minibatch number')
            plt.ylabel('minibatch loss')
            plt.show()
    return total_loss,total_correct

with tf.Session() as sess:
    with tf.device("/cpu:0"): #"/cpu:0" or "/gpu:0" 
        sess.run(tf.global_variables_initializer())
        print('Training')
        run_model(sess,y_out,mean_loss,X_train,y_train,1,64,100,train_step,True)
        print('Validation')
        run_model(sess,y_out,mean_loss,X_val,y_val,1,64)

Training


NameError: name 'X_train' is not defined